In [1]:
from typing import Union, Optional, List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

/Users/heewungsong/anaconda3/envs/kor-example/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
llm = ChatOpenAI(model="gpt-4o-mini")

class Joke(BaseModel):
    """Joke to tell user."""

    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class Response(BaseModel):
    output: Union[Joke, ConversationalResponse]


structured_llm = llm.with_structured_output(Response)

structured_llm.invoke("Tell me a joke about cats")

Response(output=Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=7))

In [19]:
structured_llm = llm.with_structured_output(Joke, method="json_mode")

structured_llm.invoke(
    "Tell me a joke about cats, respond in JSON with `setup` and `punchline` keys"
)

Joke(setup='Why was the cat sitting on the computer?', punchline='Because it wanted to keep an eye on the mouse!', rating=None)

In [4]:
class Quiz(BaseModel):
    """Korean word quiz"""

    example_sentence: str = Field(description="The example sentence")
    choices: List[str] = Field(description="The choices of the quiz")
    answer: int = Field(description="The answer index of the quiz")

In [5]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

llm_openai = ChatOpenAI(model="gpt-4o-mini")
llm_antropic = ChatAnthropic(model="claude-3-5-sonnet-20240620")

system = """다음은 "한국어 단어 퀴즈" 문제를 생성하기 위한 프롬프트입니다. 주어진 한국어 단어를 사용하여 예시 문장을 만들고, 해당 단어의 의미를 평가할 수 있는 1-4번 선택지를 작성하세요.

# 지시 사항

1. 주어진 한국어 단어를 사용하여 예시 문장을 작성하세요.
2. 해당 단어의 일반적인 의미를 고려하여 4개의 선택지를 만드세요.
   - 선택지에는 단어의 올바른 의미 1개와 오답 3개가 포함되어야 합니다.
3. 선택지 중 하나는 해당 단어의 올바른 의미를 포함하고 있어야 하며, 나머지 선택지들은 비슷하거나 전혀 관련 없는 의미로 구성해 주세요.
4. 주어진 난이도(초급, 중급, 고급)에 따라 예시 문장과 선택지의 복잡도를 조정하세요.

# 출력 형식

- 예시 문장을 먼저 제시한 후 보기 1-4를 순서대로 나열합니다.
- 예시 문장과 보기는 각각 새로운 줄에서 시작됩니다.
- 각 보기 앞에는 번호(1-4)를 붙이세요.
- 답(answer)은 1-4 중 하나의 정수로 출력해주세요.

# 예시

**입력:**
단어: ‘바쁘다’
난이도: ‘초급’

**출력:**
- 예시 문장: 오늘 하루가 너무 [바쁘다] 보니 쉬는 시간이 없었다.
- 보기:
  1. 해야 할 일이 많아 여유가 없다.
  2. 몸이 아프고 피곤하다.
  3. 날씨가 덥고 습하다.
  4. 물건이 아주 싸다.

# 주의 사항

- 퀴즈 문제의 선택지는 명확하고 간결하게 작성하세요.
- 오답 선택지는 다양한 의미를 포함하여 학습자가 혼동할 수 있도록 구성하세요.

이 프롬프트에 주어진 단어를 대입해 퀴즈 문제를 생성해 보세요!
-----

단어: {word}
난이도: {level}"""

prompt = ChatPromptTemplate.from_template(system)
structured_llm = llm_antropic.with_structured_output(Quiz, strict=True)
chain = prompt | structured_llm

In [6]:
result = chain.invoke({
    "word": "책상", #춥다, 배고프다, 덥다, 무겁다, 책상
    "level": "초급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)

학생들은 교실에서 [책상]에 앉아 공부합니다.
['글을 읽거나 쓸 때 사용하는 가구', '음식을 먹을 때 사용하는 가구', '옷을 보관하는 가구', '잠을 잘 때 사용하는 가구']
1


In [20]:
result = chain.invoke({
    "word": "무겁다",
    "level": "중급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)


짐이 너무 [무겁다] 보니 혼자서는 들 수가 없다.
['1. 가볍고 다루기 쉬운 상태이다.', '2. 물체의 질량이 크고 힘으로 들어올리기 어렵다.', '3. 기분이 우울하고 힘든 상태이다.', '4. 색깔이 어두운 상태이다.']
2


In [26]:
result = chain.invoke({
    "word": "배고프다",
    "level": "초급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)

점심을 못 먹어서 지금 너무 [배고프다].
['음식을 먹고 싶은 상태이다.', '잠을 자고 싶은 상태이다.', '화가 나서 기분이 좋지 않다.', '운동을 하고 싶은 상태이다.']
1


In [27]:
result = chain.invoke({
    "word": "배고프다",
    "level": "중급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)

장시간 운동을 한 후에 너무 [배고프다] 보니 식당에서 평소보다 두 배나 더 많이 먹었다.
['음식을 먹고 싶은 상태이다', '체력이 많이 소모되어 피곤하다', '스트레스로 인해 불안한 상태이다', '새로운 것을 배우고 싶은 욕구가 강하다']
1


In [25]:
result = chain.invoke({
    "word": "배고프다",
    "level": "고급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)

장시간의 회의로 인해 점심을 거르다 보니, 오후가 되자 배가 고파 집중력이 현저히 떨어졌다.
['음식을 섭취하지 않아 신체가 에너지를 요구하는 상태이다.', '새로운 지식이나 경험을 갈망하는 정신적 상태를 나타낸다.', '금전적으로 궁핍하여 경제적 어려움을 겪고 있다.', '특정 대상에 대한 강한 호기심이나 열망을 느끼고 있다.']
1


In [28]:
result = chain.invoke({
    "word": "이중적이다",
    "level": "고급"
})

print(result.example_sentence)
print(result.choices)
print(result.answer)

그의 행동은 매우 [이중적이다]. 겉으로는 친절하게 대하면서 뒤에서는 나쁜 소문을 퍼뜨리고 있다.
['한 가지 상황에 두 가지 상반된 태도나 성질을 보이는 것이다.', '매우 복잡하고 난해한 상태를 의미한다.', '어떤 일에 대해 신중하고 조심스러운 태도를 취하는 것이다.', '두 가지 일을 동시에 잘하는 능력을 가리킨다.']
1
